In [ ]:
# cat hosts 
[webserver]

172.31.45.28 ansible_user="ubuntu" ansible_port=22 ansible_ssh_private_key_file="ansible.pem"
root@ip-172-31-33-199:/home/ubuntu# ansible -i hosts webserver -b -m ping 
172.31.45.28 | SUCCESS => {
    "ansible_facts": {
        "discovered_interpreter_python": "/usr/bin/python3"
    },
    "changed": false,
    "ping": "pong"
}


In [ ]:
# ansible -i hosts webserver -b -m fetch -a "src=/etc/nginx/sites-available/default dest=./"
172.31.45.28 | CHANGED => {
    "changed": true,
    "checksum": "2818bd2f1cd8893161f9ebab66504eb819b68f81",
    "dest": "/home/ubuntu/172.31.45.28/etc/nginx/sites-available/default",
    "md5sum": "cea6bbf331b36b3c14d60b132f3aaa1f",
    "remote_checksum": "2818bd2f1cd8893161f9ebab66504eb819b68f81",
    "remote_md5sum": null
}
root@ip-172-31-33-199:/home/ubuntu# ls /home/ubuntu/172.31.45.28/etc/nginx/sites-available/
default


# ansible -i hosts webserver -b -m fetch -a "src=/etc/nginx/sites-available/default dest=./ flat=true"
172.31.45.28 | CHANGED => {
    "changed": true,
    "checksum": "2818bd2f1cd8893161f9ebab66504eb819b68f81",
    "dest": "/home/ubuntu/default",
    "md5sum": "cea6bbf331b36b3c14d60b132f3aaa1f",
    "remote_checksum": "2818bd2f1cd8893161f9ebab66504eb819b68f81",
    "remote_md5sum": null
}

In [ ]:
# cp -pvr default vhost.conf.tmpl
'default' -> 'vhost.conf.tmpl'


In [ ]:
# cat vhost.conf.tmpl 
server {
	listen 80;
	listen [::]:80;

	root {{doc_root}}/{{domain_name}}/public_html/;

	# Add index.php to the list if you are using PHP
	index index.html index.htm index.nginx-debian.html;

	server_name {{domain_name}} www.{{domain_name}};

	location / {
		# First attempt to serve request as file, then
		# as directory, then fall back to displaying a 404.
		try_files $uri $uri/ =404;
	}

	# pass PHP scripts to FastCGI server
	location ~ \.php$ {
		include snippets/fastcgi-php.conf;
		fastcgi_pass unix:/var/run/php/php7.4-fpm.sock;
	}

	# deny access to .htaccess files, if Apache's document root
	location ~ /\.ht {
		deny all;
	}
}


In [ ]:
# cat package.vars 
---
packages:
  - nginx
  - php7.4-fpm
  - php7.4-mysql
root@ip-172-31-33-199:/home/ubuntu# cat vhost.vars 
---
nginx_user: "www-data"
nginx_group: "www-data"
doc_root: "/var/www"
domain_name: "inenso.in"
root@ip-172-31-33-199:/home/ubuntu# cat service.vars 
---
services:
  - nginx
  - php7.4-fpm

In [ ]:
---
- name: "nginx-php Installation"
  become: yes
  hosts: webserver
  vars_files:
    - package.vars
    - vhost.vars
    - service.vars
  tasks:
    - name: "installing nginx and php packages"
      apt:
        update_cache: yes
        name: "{{ packages }}"
        state: present
   
    - name: "copying vhost to sites-available for {{ domain_name }}"
      template:
        src: "vhost.conf.tmpl"
        dest: "/etc/nginx/sites-available/{{ domain_name }}"
      register: vhost_status

    - name: "creating sym link to sites-enabled for {{ domain_name }}"
      file:
        state: link
        src: "/etc/nginx/sites-available/{{ domain_name }}"
        dest: "/etc/nginx/sites-enabled/{{ domain_name }}.conf"
    
    - name: "creating document root {{ doc_root }}/{{ domain_name }}/public_html"
      file:
        state: directory
        path: "{{ doc_root }}/{{ domain_name }}/public_html"
        owner: "{{ nginx_user }}"
        group: "{{ nginx_group }}"

    - name: "copying website to {{ domain_name }} document root"
      copy:
        src: "./website/"
        dest: "{{ doc_root }}/{{ domain_name }}/public_html"
        owner: "{{ nginx_user }}"
        group: "{{ nginx_group }}"

    - name: "nginx and php service enabling/restarting"
      when: vhost_status.changed == true
      service:
        name: "{{ item }}"
        state: restarted
        enabled: yes
      with_items: "{{ services }}"

    - name: "creating info.php"
      copy:
        content: "<?php phpinfo(); ?>"
        dest: "{{ doc_root }}/{{ domain_name }}/public_html/info.php"
        owner: "{{ nginx_user }}"
        group: "{{ nginx_group }}"

In [ ]:
# ansible-playbook -i hosts nginx-php.yml 

PLAY [nginx-php Installation] *****************************************************************************************************************************

TASK [Gathering Facts] ************************************************************************************************************************************
ok: [172.31.45.28]
ok: [172.31.39.45]

TASK [installing nginx and php packages] ******************************************************************************************************************
ok: [172.31.45.28]
ok: [172.31.39.45]

TASK [copying vhost to sites-available for inenso.in] *****************************************************************************************************
ok: [172.31.39.45]
ok: [172.31.45.28]

TASK [creating sym link to sites-enabled for inenso.in] ***************************************************************************************************
ok: [172.31.45.28]
ok: [172.31.39.45]

TASK [creating document root /var/www/inenso.in/public_html] **********************************************************************************************
ok: [172.31.45.28]
ok: [172.31.39.45]

TASK [copying website to inenso.in document root] *********************************************************************************************************
ok: [172.31.45.28]
ok: [172.31.39.45]

TASK [nginx and php service enabling/restarting] **********************************************************************************************************
skipping: [172.31.45.28] => (item=nginx) 
skipping: [172.31.45.28] => (item=php7.4-fpm) 
skipping: [172.31.39.45] => (item=nginx) 
skipping: [172.31.39.45] => (item=php7.4-fpm) 

TASK [creating info.php] **********************************************************************************************************************************
changed: [172.31.45.28]
changed: [172.31.39.45]

PLAY RECAP ************************************************************************************************************************************************
172.31.39.45               : ok=7    changed=1    unreachable=0    failed=0    skipped=1    rescued=0    ignored=0   
172.31.45.28               : ok=7    changed=1    unreachable=0    failed=0    skipped=1    rescued=0    ignored=0 

In [ ]:
$ ansible -i hosts webserver -b -m service -a "name=php7.4-fpm state=restarted"

$ ansible -i hosts webserver -b -m service -a "name=nginx state=restarted"


## extra

In [ ]:
#!/bin/bash

apt -y install software-properties-common
apt-add-repository ppa:ansible/ansible
apt update
apt install ansible
apt install python3-argcomplete
activate-global-python-argcomplete3


ansible --version
===========

apt-get install nginx -y

server {
	listen 80;
	listen [::]:80;

	server_name inenso.in www.inenso.in;

	root /var/www/inenso.in/html;
	index index.html index.php;

	location / {
		try_files $uri $uri/ =404;
	}
}


====
sudo ln –s /etc/nginx/sites-available/test_domain.com /etc/nginx/sites-enabled
====
server {
    listen 80 default_server;
    listen [::]:80 default_server;
   
    root /var/www/html;
    index index.html index.htm index.nginx-debian.html;

    server_name _;

    location / {
        try_files $uri $uri/ =404;
    }
}
====
# cat /etc/nginx/nginx.conf |grep -v "#"
user www-data;
worker_processes auto;
pid /run/nginx.pid;
include /etc/nginx/modules-enabled/*.conf;

events {
	worker_connections 768;
}

http {

root /var/www/html;
	sendfile on;
	tcp_nopush on;
	tcp_nodelay on;
client_max_body_size 2G;
	keepalive_timeout 65;
	types_hash_max_size 2048;


	include /etc/nginx/mime.types;
	default_type application/octet-stream;


	ssl_prefer_server_ciphers on;


	access_log /var/log/nginx/access.log;
	error_log /var/log/nginx/error.log;


	gzip on;



	include /etc/nginx/conf.d/*.conf;
	include /etc/nginx/sites-enabled/*;
}


=====================


location ~ \.php$ {
        include snippets/fastcgi-php.conf;
        fastcgi_pass unix:/var/run/php/php7.4-fpm.sock;
     }


     <?php phpinfo(); ?>

=======
[webserver]

172.31.45.28 ansible_user="ubuntu" ansible_port=22 ansible_ssh_private_key_file="ansible.pem"

[amazon]

172.31.45.28 ansible_user="ubuntu" ansible_port=22 ansible_ssh_private_key_file="ansible.pem"

=====
apt-get install php7.4-fpm php7.4-cli php7.4-mysql php7.4-curl php7.4-json -y